In [1]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import torch.nn as nn
import pandas as pd
import numpy
from transformers import AutoModel, AutoTokenizer, BertForSequenceClassification

c:\Users\Francesca\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Francesca\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\Francesca\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
df = pd.read_csv("subtaskA_train.csv")

In [4]:
X = df["comment_text"]
y = df["conspiratorial"]

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size = 0.25, random_state = 10)

In [6]:
bert_ita= "dbmdz/bert-base-italian-cased"
tokenizer = AutoTokenizer.from_pretrained(bert_ita)
#model = AutoModel.from_pretrained(bert_ita)

In [17]:
# define model
class BERT_classif(nn.Module):
    def __init__(self, bert):
        super(BERT_classif, self).__init__()
        '''
        self.num_classes = num_classes #number of classes
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length
        '''
        in_size = X_train.shape
        hid = (len(list(X_train)))/(2*( in_size + 2))
        self.lstm = nn.LSTM(input_size=in_size, hidden_size=hid,
                        num_layers=1) #lstm
        self.encoder = BertForSequenceClassification.from_pretrained(bert_ita) #bert encoder
        self.fc_1 =  nn.Linear(hidden_size, 128) #fully connected 1
        self.fc = nn.Linear(128, 2) #fully connected last layer

        self.relu = nn.ReLU()
    
    def forward(self,x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) #hidden state
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) #internal state
        # Propagate input through LSTM
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
        hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
        out = self.relu(hn)
        out = self.fc_1(out) #first Dense
        out = self.relu(out) #relu
        out = self.fc(out) #Final Output
        return out

In [23]:
bert_classifier = BERT_classif(bert_ita) #our lstm class 
criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(bert_classifier.parameters(), lr=0.9) 

TypeError: can only concatenate tuple (not "int") to tuple

In [21]:
for epoch in range(100):
  outputs = bert_classifier.forward(X_train_tensors_final) #forward pass
  optimizer.zero_grad() #caluclate the gradient, manually setting to 0
 
  # obtain the loss function
  loss = criterion(outputs, y_train_tensors)
 
  loss.backward() #calculates the loss of the loss function
 
  optimizer.step() #improve from loss, i.e backprop
  if epoch % 100 == 0:
    print("Epoch: %d, loss: %1.5f" % (epoch, loss.item())) 

NameError: name 'bert_classifier' is not defined